In [1]:
from huggingface_hub import snapshot_download

# Hugging Face Hub에 있는 kakaocorp/kanana-1.5-15.7b-a3b-instruct 모델
model_id = "kakaocorp/kanana-1.5-15.7b-a3b-instruct"

# 전체 스냅샷(모든 파일: 모델 가중치, config, tokenizer 등)을 ./model/kakao 폴더에 다운로드
snapshot_download(repo_id=model_id,
                  local_dir="./model/kakao",
                  local_dir_use_symlinks=False,
                  revision="main")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:982: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 21 files:   0%|          | 0/21 [00:00<?, ?it/s]

LICENSE: 0.00B [00:00, ?B/s]

README.md: 0.00B [00:00, ?B/s]

kanana-logo.png:   0%|          | 0.00/59.0k [00:00<?, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

'/content/model/kakao'

In [2]:
# gguf 변환 및 양자화를 위해 설치합니다.
!git clone https://github.com/ggerganov/llama.cpp.git

# GGUF란? (GPT-Generated Unified Format)
## llama.cpp 프로젝트에서 사용하는 최신 모델 파일 포맷이다.
## GGUF는 모델 가중치뿐 아니라 토크나이저 정보, 메타데이터까지 하나로 묶어두는 효율적인 포맷

# 양자화란? (Quantization)
## 양자화는 모델 가중치를 원래 float32(32비트 실수) 대신 더 작은 정밀도 (예: 16비트, 8비트, 4비트 정수)로 변환하는 과정
## 메모리 사용량이 줄어들고, 추론 속도가 빨라진다.
## 정확도 손실이 있을 수 있다.

Cloning into 'llama.cpp'...
remote: Enumerating objects: 60574, done.
remote: Counting objects: 100% (236/236), done.
remote: Compressing objects: 100% (149/149), done.
remote: Total 60574 (delta 162), reused 87 (delta 87), pack-reused 60338 (from 3)
Receiving objects: 100% (60574/60574), 151.01 MiB | 23.31 MiB/s, done.
Resolving deltas: 100% (43926/43926), done.


In [2]:
import os

print(os.getcwd())  # 현재 작업 디렉토리 출력
os.chdir("./llama.cpp") # 작업 디렉토리 이동
print(os.getcwd())

/content
/content/llama.cpp


In [3]:
# 설치가 다 완료되지 않을 시, 윗 셀부터 다시 시작
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/nightly, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/nightly, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/nightly
Ignoring torch: markers 'platform_machine == "s390x"' don't match your environment
Ignoring torch: markers 'platform_machine == "s390x"' don't match your environment


In [4]:
# llama.cpp 안에 빌드 전용 디렉토리를 새로 만든다.
## (소스 코드와 빌드 결과물 분리하기 위해서)
!mkdir build

In [6]:
# 작업 디렉토리 이동
os.chdir("./build")
print(os.getcwd())

/content/llama.cpp/build


In [7]:
# CMakeLists.txt 파일을 읽어서 빌드 설정 파일 생성 / 컴파일 환경 준비
!cmake ..

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMAKE_BUILD_TYPE=Release
-- Found Git: /usr/bin/git (found version "2.34.1")
-- The ASM compiler identification is GNU
-- Found assembler: /usr/bin/cc
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE
-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- GGML_SYSTEM_ARCH: x86
-- Including CPU backend
-- Found OpenMP_C: 

In [8]:
# 빌드 설정을 실제로 실행해서 컴파일
## --config Release 는 최적화 모드
!cmake --build . --config Release

[  1%] Building C object ggml/src/CMakeFiles/ggml-base.dir/ggml.c.o
[  1%] Building CXX object ggml/src/CMakeFiles/ggml-base.dir/ggml.cpp.o
[  2%] Building C object ggml/src/CMakeFiles/ggml-base.dir/ggml-alloc.c.o
[  2%] Building CXX object ggml/src/CMakeFiles/ggml-base.dir/ggml-backend.cpp.o
[  2%] Building CXX object ggml/src/CMakeFiles/ggml-base.dir/ggml-opt.cpp.o
[  3%] Building CXX object ggml/src/CMakeFiles/ggml-base.dir/ggml-threading.cpp.o
[  3%] Building C object ggml/src/CMakeFiles/ggml-base.dir/ggml-quants.c.o
[  4%] Building CXX object ggml/src/CMakeFiles/ggml-base.dir/gguf.cpp.o
[  4%] Linking CXX shared library ../../bin/libggml-base.so
[  4%] Built target ggml-base
[  4%] Building C object ggml/src/CMakeFiles/ggml-cpu.dir/ggml-cpu/ggml-cpu.c.o
[  5%] Building CXX object ggml/src/CMakeFiles/ggml-cpu.dir/ggml-cpu/ggml-cpu.cpp.o
[  5%] Building CXX object ggml/src/CMakeFiles/ggml-cpu.dir/ggml-cpu/repack.cpp.o
[  6%] Building CXX object ggml/src/CMakeFiles/ggml-cpu.dir/ggml-

In [10]:
os.chdir("..")
print(os.getcwd())

/content/llama.cpp


In [15]:
# gguf 파일을 담기 위한 폴더를 만든다.
!mkdir ../model_gguf/kakao

In [16]:
# Hugging Face에서 받은 모델을 llama.cpp에서 읽을 수 있는 GGUF 포맷으로 변환
!python convert_hf_to_gguf.py ../model/kakao/ \
  --outfile ../model_gguf/kakao/kanana-1.5-15.7b-a3b-instruct.gguf \
  --outtype f16  ##f32

INFO:hf-to-gguf:Loading model: kakao
INFO:hf-to-gguf:Model architecture: MixtralForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00007.safetensors'
INFO:hf-to-gguf:blk.1.ffn_gate_inp.weight,   torch.bfloat16 --> F32, shape = {2048, 64}
INFO:hf-to-gguf:blk.1.ffn_norm.weight,       torch.bfloat16 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.10.attn_v.weight,        torch.bfloat16 --> F16, shape = {2048, 1024}
INFO:hf-to-gguf:blk.11.ffn_norm.weight,      torch.bfloat16 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.13.ffn_gate_inp.weight,  torch.bfloat16 --> F32, shape = {2048, 64}
INFO:hf-to-gguf:blk.15.ffn_norm.weight,      torch.bfloat16 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.17.ffn_norm.weight,      torch.bfloat16 --> F32, shape = {2048}
INFO:hf-to-gguf:blk.19.attn_k.weight,      

In [17]:
# 양자화 수행
## Q4_K_M 으로 양자화 (4비트 양자화 + KMeans)
## 그 외에도 Q4_0 / Q8_0 등 다양한 방법이 있다.
## 만약 용량 문제가 발생하면, 이미 model_gguf 폴더가 있으므로 model 폴더 안의 모델을 지울 수 있다.
!./build/bin/llama-quantize \
  ../model_gguf/kakao/kanana-1.5-15.7b-a3b-instruct.gguf \
  ../model_gguf/kakao/kanana-1.5-15.7b-a3b-instruct-q4_k_m.gguf \
  Q4_K_M

main: build = 6341 (b66df9d9)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04.2) 11.4.0 for x86_64-linux-gnu
main: quantizing '../model_gguf/kakao/kanana-1.5-15.7b-a3b-instruct.gguf' to '../model_gguf/kakao/kanana-1.5-15.7b-a3b-instruct-q4_k_m.gguf' as Q4_K_M
llama_model_loader: loaded meta data with 36 key-value pairs and 323 tensors from ../model_gguf/kakao/kanana-1.5-15.7b-a3b-instruct.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Kakao
llama_model_loader: - kv   3:                         general.size_label str              = 64x1.4B
llama_model_loader: - kv   4:                            general.license str      

In [18]:
# gguf 파일 생성 완료/ 올라마에 서빙하기 위한 준비 완료